In [2]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
#Get the dataset
dataset = pd.read_csv('loan.csv')

In [4]:
#Deal the married column
# Mode to fill the missing values
dataset['Married'] = dataset['Married'].fillna(dataset['Married'].mode()[0])

In [5]:
#Deal with Education Column
#Change the Education to numeric values
dataset['Education'] = dataset['Education'].map({'Graduate': 1, 'Not Graduate': 0})
dataset

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,1,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,1,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,1,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,0,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,1,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,1,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,1,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,1,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,1,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [6]:
## Get cwhether there is coapplicant income
CoApplicantIncome_binary=[]
value=0
for data in dataset['CoapplicantIncome']:
    if data == 0:
        value = 0
    elif data>0:
        value =1
    else:
        print("There is a weird value in the data")
    CoApplicantIncome_binary.append(value)
dataset['CoApplicantIncome_binary'] = CoApplicantIncome_binary 

In [7]:
# Get Loan Amount Binary
LoanAmount_binary=[]
value=0
for data in dataset['LoanAmount']:
    if data == 0:
        value = 0
    elif pd.isna(data):
        value = 0
    elif data>0:
        value =1
    else:
        print(f"There is a weird value in the data.")
    LoanAmount_binary.append(value)

dataset['LoanAmount_binary'] = LoanAmount_binary

In [8]:
#Fill in missing loan amount values 
dataset['LoanAmount']= dataset['LoanAmount'].fillna(dataset['LoanAmount'].median())

In [9]:
# Get true values 
dataset['LoanAmount'] = dataset['LoanAmount']*1000
dataset

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,CoApplicantIncome_binary,LoanAmount_binary
0,LP001002,Male,No,0,1,No,5849,0.0,128000.0,360.0,1.0,Urban,Y,0,0
1,LP001003,Male,Yes,1,1,No,4583,1508.0,128000.0,360.0,1.0,Rural,N,1,1
2,LP001005,Male,Yes,0,1,Yes,3000,0.0,66000.0,360.0,1.0,Urban,Y,0,1
3,LP001006,Male,Yes,0,0,No,2583,2358.0,120000.0,360.0,1.0,Urban,Y,1,1
4,LP001008,Male,No,0,1,No,6000,0.0,141000.0,360.0,1.0,Urban,Y,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,1,No,2900,0.0,71000.0,360.0,1.0,Rural,Y,0,1
610,LP002979,Male,Yes,3+,1,No,4106,0.0,40000.0,180.0,1.0,Rural,Y,0,1
611,LP002983,Male,Yes,1,1,No,8072,240.0,253000.0,360.0,1.0,Urban,Y,1,1
612,LP002984,Male,Yes,2,1,No,7583,0.0,187000.0,360.0,1.0,Urban,Y,0,1


In [10]:
# Get Acceptible DTI column
def calculate_monthly_payment(P, annual_interest_rate, n):
    """
    Calculate the monthly loan payment based on loan amount, interest rate, and loan term.

    Parameters:
    P (float): Loan amount (in actual value, not thousands)
    annual_interest_rate (float): Annual interest rate (e.g., 0.1 for 10%)
    n (int): Loan term in months

    Returns:
    float: Monthly loan payment
    """
    # Convert annual interest rate to monthly interest rate
    r = annual_interest_rate / 12  
    
    # Avoid division by zero if interest rate is 0 (special case for zero-interest loans)
    if r == 0:
        return P / n
    
    # Loan payment formula
    M = P * (r * (1 + r)**n) / ((1 + r)**n - 1)
    return M

monthly_payment = []
for loan_amount, loan_term in zip(dataset['LoanAmount'], dataset['Loan_Amount_Term']):
    payment = calculate_monthly_payment(loan_amount, 0.1, loan_term)
    monthly_payment.append(round(payment, 2))
dataset['MonthlyPayment'] = monthly_payment


In [11]:
dataset['CombinedIncome'] = dataset['ApplicantIncome'] + dataset['CoapplicantIncome']

In [12]:
# Get Acceptable DTI
DTI = []
for monthly_payment, combined_income in zip(dataset['MonthlyPayment'], dataset['CombinedIncome']):
    DTI_value= (monthly_payment/combined_income) * 100
    DTI.append(round(DTI_value, 2))
    
dataset['DTI'] = DTI

acceptable_DTI = []
for dti in dataset['DTI']:
    if dti < 43:
        acceptable_DTI.append(1)
    else:
        acceptable_DTI.append(0)
dataset['Acceptable_DTI'] = acceptable_DTI

In [13]:
dataset.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status',
       'CoApplicantIncome_binary', 'LoanAmount_binary', 'MonthlyPayment',
       'CombinedIncome', 'DTI', 'Acceptable_DTI'],
      dtype='object')

In [17]:
modelling_data = dataset[['Married', 'Education', 'CoApplicantIncome_binary', 'LoanAmount_binary', 'Acceptable_DTI', 'Property_Area', 'Loan_Status']]
modelling_data

,Married,Education,CoApplicantIncome_binary,LoanAmount_binary,Acceptable_DTI,Property_Area,Loan_Status
0,No,1,0,0,1,Urban,Y
1,Yes,1,1,1,1,Rural,N
2,Yes,1,0,1,1,Urban,Y
3,Yes,0,1,1,1,Urban,Y
4,No,1,0,1,1,Urban,Y
...,...,...,...,...,...,...,...
609,No,1,0,1,1,Rural,Y
610,Yes,1,0,1,1,Rural,Y
611,Yes,1,1,1,1,Urban,Y
612,Yes,1,0,1,1,Urban,Y


In [28]:
#Dealing with categorical values 
modelling_data = pd.get_dummies(modelling_data, columns=['Property_Area'], drop_first=True)
modelling_data = pd.get_dummies(modelling_data, columns=['Loan_Status'], drop_first=True)
modelling_data = pd.get_dummies(modelling_data, columns=['Married'], drop_first=True)

In [29]:
modelling_data

,Education,CoApplicantIncome_binary,LoanAmount_binary,Acceptable_DTI,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_Y,Married_Yes
0,1,0,0,1,False,True,True,False
1,1,1,1,1,False,False,False,True
2,1,0,1,1,False,True,True,True
3,0,1,1,1,False,True,True,True
4,1,0,1,1,False,True,True,False
...,...,...,...,...,...,...,...,...
609,1,0,1,1,False,False,True,False
610,1,0,1,1,False,False,True,True
611,1,1,1,1,False,True,True,True
612,1,0,1,1,False,True,True,True


In [30]:
# Splitting features (X) and target variable (y)
X = modelling_data.iloc[:, :7]  # Select all columns except the first one (Survived)
y = modelling_data.iloc[:, -1]  # Select the first column (Survived)


In [31]:
# Splitting the dataset into training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [32]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [33]:
#Fitting kernel SVM to the training set 
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state=0, C = 1, gamma = 0.5, degree = 2)
classifier.fit(X_train, y_train)

SVC(C=1, degree=2, gamma=0.5, random_state=0)

In [36]:
y_pred = classifier.predict(X_test)


array([ True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True, False,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True, False,  True,
       False,  True,  True, False,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,

In [38]:
from sklearn.metrics import confusion_matrix, classification_report

# Compute Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

# Display Classification Report
print("Classification Report:\n", classification_report(y_test, y_pred))

Confusion Matrix:
 [[15 49]
 [ 5 85]]
Classification Report:
               precision    recall  f1-score   support

       False       0.75      0.23      0.36        64
        True       0.63      0.94      0.76        90

    accuracy                           0.65       154
   macro avg       0.69      0.59      0.56       154
weighted avg       0.68      0.65      0.59       154



In [ ]:
#Implementing Grid Search
from sklearn.model_selection import GridSearchCV
parameters = [
    {"C":[1,10,100,1000], 'kernel':['linear']},
    {"C":[1,10,100,1000], 'kernel':['rbf'], 'gamma':[0.5, 0.1, 0.01, 0.001, 0.0001]}
    ]
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)